In [1]:
import geopandas as gpd
import fiona
import matplotlib.pyplot as plt 
import pandas as pd 
import glob
import os

gdb_path = rf'D:\Datasets\Illegal Fishing\Original Data\WDPA_Jun2024_Public.gdb'

layers = fiona.listlayers(gdb_path)
print("layers =", layers)



layers = ['WDPA_poly_Jun2024', 'WDPA_point_Jun2024', 'WDPA_source_Jun2024']


Loading data about marine protected areas around the world which are in two formats, point and polygons

In [2]:
layers =  ['WDPA_poly_Jun2024', 'WDPA_point_Jun2024', 'WDPA_source_Jun2024']
poly_data = gpd.read_file(gdb_path, layer=layers[0])
print(poly_data.head())

c:\Users\gurpr\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


        WDPAID   WDPA_PID PA_DEF                                        NAME  \
0  555645336.0  555645336      1                                Pangasananan   
1  555645495.0  555645495      1                             Inuyog No Banwa   
2  555637442.0  555637442      1  Sombrero Island Nature Reserve Marine Park   
3      32637.0      32637      1                                Sandy Island   
4  555705843.0  555705843      1                                  Little Bay   

                                    ORIG_NAME  \
0                                Pangasananan   
1                             Inuyog No Banwa   
2  Sombrero Island Nature Reserve Marine Park   
3                                Sandy Island   
4                                  Little Bay   

                                              DESIG  \
0                                      Pangasananan   
1                                   Inuyog No Banwa   
2  Ramsar Site, Wetland of International Importance   
3   

In [10]:
point_data = gpd.read_file(gdb_path, layer=layers[1])
print(point_data.head())
print(len(point_data))
print(point_data['STATUS'])
print(point_data['STATUS_YR'])

        WDPAID   WDPA_PID PA_DEF                            NAME  \
0     315531.0     315531      1                           Susui   
1       7623.0       7623      1                            Kyai   
2  555547651.0  555547651      1                          Wahere   
3       6702.0       6702      1                     Miani Dhand   
4  555547652.0  555547652      1  Abalolo, Gwaedalo, Ailau (AGA)   

                        ORIG_NAME                       DESIG  \
0                           Susui                Not Reported   
1                            Kyai              Forest Reserve   
2                          Wahere  Marine Protected Area/Tabu   
3                     Miani Dhand          Wildlife Sanctuary   
4  Abalolo, Gwaedalo, Ailau (AGA)  Marine Protected Area/Tabu   

                    DESIG_ENG      DESIG_TYPE      IUCN_CAT        INT_CRIT  \
0                Not Reported        National            VI  Not Applicable   
1              Forest Reserve        Natio

Loading Ocean boundary data and filtering out any areas not in the ocean

In [6]:
gdb_path_ocean = rf'D:\Datasets\Illegal Fishing\Original Data\ne_110m_ocean\ne_110m_ocean.shp'

ocean_data = gpd.read_file(gdb_path_ocean)
print(ocean_data)

print(ocean_data.columns)


   scalerank featurecla  min_zoom  \
0          0      Ocean       0.0   
1          0      Ocean       0.0   

                                            geometry  
0  POLYGON ((49.11029 41.28229, 48.58447 41.8089,...  
1  POLYGON ((180 71.51571, 178.7253 71.0988, 178....  
Index(['scalerank', 'featurecla', 'min_zoom', 'geometry'], dtype='object')


In [ ]:

ocean_data = ocean_data.to_crs(poly_data.crs)
point_data = point_data.to_crs(poly_data.crs)

# Create spatial index for ocean boundaries
ocean_sindex = ocean_data.sindex

# Function to check if a geometry intersects with any of the ocean boundaries
def intersects_ocean(geom, ocean_gdf, ocean_sindex):
    # Find potential matches using spatial index
    possible_matches_index = list(ocean_sindex.intersection(geom.bounds))
    possible_matches = ocean_gdf.iloc[possible_matches_index]
    
    # Check for actual intersection
    return possible_matches.intersects(geom).any()

# Filter polygons that intersect with the ocean boundaries
poly_data['within_ocean'] = poly_data.geometry.apply(lambda geom: intersects_ocean(geom, ocean_data, ocean_sindex))
poly_within_ocean = poly_data[poly_data['within_ocean']]

# Filter points that intersect with the ocean boundaries
point_data['within_ocean'] = point_data.geometry.apply(lambda point: intersects_ocean(point, ocean_data, ocean_sindex))
points_within_ocean = point_data[point_data['within_ocean']]

# Display the filtered data
print("Filtered Polygons:")
print(poly_within_ocean)
print("\nFiltered Points:")
print(points_within_ocean)

poly_within_ocean.to_file(rf'D:\Datasets\Illegal Fishing\Processed Data\Filtered MPA data\filtered_polygons.shp')
points_within_ocean.to_file(rf'D:\Datasets\Illegal Fishing\Processed Data\Filtered MPA data\filtered_points.shp')


Loading up the vessel jouney data and only keeping journeys that were in the ocean.

In [ ]:
csv_files = glob.glob(rf'D:\Datasets\Illegal Fishing\Original Data\Ship data\*.csv')

df_list = []
# Loop through each file and add the shiptype column
for file in csv_files:

    df = pd.read_csv(file)
    shiptype = os.path.basename(file).replace('.csv', '')

    df['shiptype'] = shiptype
    df_list.append(df)

ship_data = pd.concat(df_list, ignore_index=True)

ship_data.to_csv(rf'D:\Datasets\Illegal Fishing\Processed Data\Complete_ship_data.csv')


In [ ]:
file_path = rf'D:\Datasets\Illegal Fishing\Processed Data\Complete_ship_data.csv'

df = pd.read_csv(file_path)
print(df.columns )
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)
df.to_csv(file_path, index=False)
print(df.columns)

Plotting the data

In [ ]:
source_data = gpd.read_file(gdb_path, layer=layers[2])
print(source_data.head())

In [ ]:

point_data.plot()
plt.show()

In [ ]:
poly_data['geometry'] = poly_data['geometry'].simplify(tolerance=0.01, preserve_topology=True)
fig, ax = plt.subplots(figsize=(10, 10))
poly_data.plot(ax=ax, color='blue', edgecolor='black')

plt.title('Multipolygon Data Plot')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()